# 參考步驟自: [LeeMeng大的進擊的BERT：NLP界的巨人之力與遷移學習](https://leemeng.tw/attack_on_bert_transfer_learning_in_nlp.html)

# Step1. 準備原始文本數據

In [1]:
import pandas as pd

### 原始資料 = df

In [2]:
df = pd.read_csv("/content/drive/Shareddrives/Media Framing/datasets/0_datasets.csv", encoding = "utf-8")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24763 entries, 0 to 24762
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Headline  24763 non-null  object
 1   Link      24763 non-null  object
 2   Date      24763 non-null  object
 3   Source    24763 non-null  object
 4   Category  24763 non-null  object
dtypes: object(5)
memory usage: 967.4+ KB


### 割盲腸資料 = df_short
- 作者想把過長樣本剔除，以避免BERT無法將整個資料放到記憶體不多的GPU
- 我先觀察有多少筆>30筆的資料

In [3]:
MAX_LENGTH = 30
df_short = df[~(df.Headline.apply(lambda x : len(x)) > MAX_LENGTH)]
df_short.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19530 entries, 0 to 24762
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Headline  19530 non-null  object
 1   Link      19530 non-null  object
 2   Date      19530 non-null  object
 3   Source    19530 non-null  object
 4   Category  19530 non-null  object
dtypes: object(5)
memory usage: 915.5+ KB


In [4]:
df_short.tail(20)

,Headline,Link,Date,Source,Category
24743,HBL／松山新星林現惟喊超越高國豪 期待對決陳將双,/news/story/8688/5901097?from=udn-catebreaknew...,2021-11-19 00:56,聯合新聞,sport
24744,HBL／看書找人生方向 陳將双喊奪冠目標,/news/story/8688/5900914?from=udn-catebreaknew...,2021-11-18 21:30,聯合新聞,sport
24745,HBL／歷史不重演 東泰打下成功報仇,/news/story/8688/5900742?from=udn-catebreaknew...,2021-11-18 19:59,聯合新聞,sport
24746,HBL／史魯齊被針對 林子皓準大三元救能仁,/news/story/8688/5900514?from=udn-catebreaknew...,2021-11-18 18:22,聯合新聞,sport
24747,HBL／最強國中生加入神盾軍 林現惟：目標超越高國豪,/news/story/8688/5900329?from=udn-catebreaknew...,2021-11-18 17:09,聯合新聞,sport
24748,HBL／泰山球衣背號有密碼 張祐瑋不知自己是王牌,/news/story/8688/5900213?from=udn-catebreaknew...,2021-11-18 16:50,聯合新聞,sport
24749,HBL／衛冕軍泰山破百迎首勝 教頭喊不滿意,/news/story/8688/5899701?from=udn-catebreaknew...,2021-11-18 15:27,聯合新聞,sport
24750,HBL／執教首戰就碰母校 高苑教頭贏球心情卻複雜,/news/story/8688/5899623?from=udn-catebreaknew...,2021-11-18 13:00,聯合新聞,sport
24751,HBL／30人剩10人 「藍魔鬼」新榮最終賽季不留遺憾,/news/story/8688/5898434?from=udn-catebreaknew...,2021-11-18 12:40,聯合新聞,sport
24752,HBL／光復再成冠軍熱門 松山高中少雙塔有全新面貌,/news/story/8688/5897988?from=udn-catebreaknew...,2021-11-18 01:17,聯合新聞,sport


### 作者抓1%的資料，約2657筆
### 我們抓大約的資料量，大概15%的資料來訓練

In [5]:
SAMPLE_FRAC = 0.15
df_short = df_short.sample(frac = SAMPLE_FRAC, random_state = 9527)

### 只取我們要的欄位

In [6]:
df_short = df_short.loc[:, ["Headline", "Source", "Category"]]

### 另存成tsv檔供PyTorch使用

In [7]:
df_short.to_csv("/content/drive/Shareddrives/Media Framing/datasets/0_datasets_short.tsv", sep = "\t", index = False, encoding = "utf-8")

print("訓練樣本數:", len(df_short))
df_short.head()

訓練樣本數: 2930


,Headline,Source,Category
3872,外資報告》摩根大通喊加碼聯發科 目標價1200元,自由時報,economy
4148,今年第14起石虎路殺 西濱後龍段石虎被撞到血肉模糊,自由時報,society
21664,開始刷存在感？這些行為看出12星座想復合,聯合新聞,life
7944,屏東潮州「賽神蝦」活動 347公克巨大泰國蝦破紀錄奪冠,公視,local
1200,警告北約 俄稱或被迫部署中程核飛彈,中時,chinese


In [8]:
df_short["Category"].value_counts()

life             776
economy          408
international    352
politics         317
sport            225
society          222
local            171
health           165
entertainment    152
chinese          136
armament           6
Name: Category, dtype: int64

# Step2. 將原始文本轉換成BERT相容的輸入格式

In [9]:
# 下載BERT
%%bash
pip install transformers tqdm boto3 requests regex -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [10]:
import torch
from transformers import BertTokenizer
from IPython.display import clear_output

PERTRAINED_MODEL_NAME = "bert-base-chinese" # 指定繁簡中文 BERT-BASE 預訓練模型

# 取得此預訓練模型所使用的tokenizer
tokenizer = BertTokenizer.from_pretrained(PERTRAINED_MODEL_NAME)

clear_output()
print("PyTorch版本:", torch.__version__)

PyTorch版本: 1.10.0+cu111


In [ ]:
import pandas as pd
path = "/content/drive/Shareddrives/Media Framing/datasets/0_datasets_short.tsv"
# 因為檔案是tsv檔，是用\t隔開，所以需要增加「delimeter」參數，才不會出錯
df_short = pd.read_csv(path, encoding = "utf-8", delimiter="\t")
df_short

,Headline,Source,Category
0,外資報告》摩根大通喊加碼聯發科 目標價1200元,自由時報,economy
1,今年第14起石虎路殺 西濱後龍段石虎被撞到血肉模糊,自由時報,society
2,開始刷存在感？這些行為看出12星座想復合,聯合新聞,life
3,屏東潮州「賽神蝦」活動 347公克巨大泰國蝦破紀錄奪冠,公視,local
4,警告北約 俄稱或被迫部署中程核飛彈,中時,chinese
...,...,...,...
2925,僵化的「醫療機關指引」七大程序，導致日本疫苗接種率僅3％,關鍵評論網,international
2926,台中比特犬數目全國第一 議員：攻擊性犬應加強列管,聯合新聞,life
2927,197縣道改善 泥水溪橋截彎取直少一半路程,中時,life
2928,國會聯絡人確診 立院協商是否停會,華視新聞,politics


實作一個可以用來讀取訓練 / 測試集的 Dataset，這是你需要徹底了解的部分。
此 Dataset 每次將 tsv 裡的一筆成對句子轉換成 BERT 相容的格式，並回傳 3 個 tensors：
- tokens_tensor：兩個句子合併後的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用來識別兩個句子界限的 binary tensor
- label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None

### `torch.utils.data`[教學](https://rowantseng.medium.com/pytorch-%E8%87%AA%E5%AE%9A%E7%BE%A9%E8%B3%87%E6%96%99%E9%9B%86-custom-dataset-7f9958a8ff15)
- 中文版 https://blog.csdn.net/weixin_41560402/article/details/108121344


In [11]:
label_table = {
    "life" : 1,
    "economy" : 2,
    "international" : 3,
    "politics" : 4,
    "sport" : 5,
    "society" : 6,
    "local" : 7,
    "health" : 8,
    "entertainment" : 9,
    "chinese" : 10,
    "armament" : 11
}

In [ ]:
from torch.utils.data import Dataset

class media_framing_datasets(Dataset):
    # 讀取前處理後的tsv檔並初始化一些參數
    def __init__(self, mode, tokenizer):
        assert mode in ["train", "test"] # 一般訓練你會需要dev set
        self.mode = mode
        # 大數據你會需要用 iterator = True
        self.df = pd.read_csv(path, sep = "\t").fillna("")
        self.len = len(self.df)
        self.label_map = label_table
        self.tokenizer = tokenizer # 我們將使用 BERT tokenizer

    # 定義回傳一筆訓練/測試數據的函式
    # 給予取得資料集裡資料的變數還有標籤的變數；可利用它完成資料預處理
    def __getitem__(self, idx):
        if self.mode == "test":
            Headline, Source = self.df.iloc[idx, :2].values
            label_tensor = None
        else:
            Headline, Source, Category = self.df.iloc[idx, :].values
            # 將 label 文字也轉換成索引方便轉換成 tensor
            label_id = self.label_map[label]
            label_tensor = torch.tensor(label_id)
            
        # 建立第一個句子的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ["[CLS]"]
        tokens_a = self.tokenizer.tokenize(Headline)
        word_pieces += tokens_a + ["[SEP]"]
        len_a = len(word_pieces)
        
        # 第二個句子的 BERT tokens
        tokens_b = self.tokenizer.tokenize(Source)
        word_pieces += tokens_b + ["[SEP]"]
        len_b = len(word_pieces) - len_a
        
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        # 將第一句包含 [SEP] 的 token 位置設為 0，其他為 1 表示第二句
        segments_tensor = torch.tensor([0] * len_a + [1] * len_b, 
                                        dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor)#, label_tensor)
    
    def __len__(self):
    # 資料集的size
        return self.len
    
    
# 初始化一個專門讀取訓練樣本的 Dataset，使用中文 BERT 斷詞
trainset = media_framing_datasets("train", tokenizer=tokenizer)

In [ ]:
# 選擇第一個樣本
sample_idx = 0

# 將原始文本拿出做比較
text_a, text_b, label = trainset.df.iloc[sample_idx].values

# 利用剛剛建立的 Dataset 取出轉換後的 id tensors
tokens_tensor, segments_tensor, label_tensor = trainset[sample_idx]

# 將 tokens_tensor 還原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = "".join(tokens)

# 渲染前後差異，毫無反應就是個 print。可以直接看輸出結果
print(f"""[原始文本]
新聞標題：{text_a}
新聞來源：{text_b}
分類  ：{label}

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：{tokens_tensor}

segments_tensor：{segments_tensor}

label_tensor   ：{label_tensor}

--------------------

[還原 tokens_tensors]
{combined_text}
""")

[原始文本]
新聞標題：外資報告》摩根大通喊加碼聯發科 目標價1200元
新聞來源：自由時報
分類  ：economy

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：tensor([ 101, 1912, 6536, 1841, 1440,  518, 3040, 3418, 1920, 6858, 1591, 1217,
        4826, 5474, 4634, 4906, 4680, 3560, 1019, 8552, 1039,  102, 5632, 4507,
        3229, 1841,  102])

segments_tensor：tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 1])

label_tensor   ：2

--------------------

[還原 tokens_tensors]
[CLS]外資報告》摩根大通喊加碼聯發科目標價1200元[SEP]自由時報[SEP]



### PySnooper: 解析作者定義的Dataset

In [ ]:
# !pip install pysnooper -q
# import pysnooper

# class media_framing_datasets(Dataset):
#         ...
#     @pysnooper.snoop()  # 加入以了解所有轉換過程
#     def __getitem__(self, idx):
#         ...

## 製作mini-batch
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 吃我們上面定義的 `media_framing_datasets`，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)

這段程式碼是你要實際將 mini-batch 丟入 BERT 做訓練以及預測的關鍵

DataLoader: 把上面我們自定義的Dataset封裝成一個迭代器
- 它可以依序輸出`Dataset`的內容
- 可同時處理shuffle(隨機洗牌)、資料校對等等的處理

In [ ]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 這個函式的輸入 `samples` 是一個 list，裡頭的每個 element 都是
# 剛剛定義的 `FakeNewsDataset` 回傳的一個樣本，每個樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor(我們不需要)
# 它會對前兩個 tensors 作 zero padding，並產生前面說明過的 masks_tensors

def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 測試集有 labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列長度
    tokens_tensors = pad_sequence(tokens_tensors, 
                    batch_first=True)
    segments_tensors = pad_sequence(segments_tensors, 
                     batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids


# 初始化一個每次回傳 64 個訓練樣本的 DataLoader
# 利用 `collate_fn` 將 list of samples 合併成一個 mini-batch 是關鍵
BATCH_SIZE = 64
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, 
              collate_fn=create_mini_batch)
# trainset: 封裝的對象

In [ ]:
data = next(iter(trainloader))

tokens_tensors, segments_tensors, \
    masks_tensors, label_ids = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
segments_tensors.shape = {segments_tensors.shape}
{segments_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")


tokens_tensors.shape   = torch.Size([64, 38]) 
tensor([[ 101, 1912, 6536,  ...,    0,    0,    0],
        [ 101,  791, 2399,  ...,    0,    0,    0],
        [ 101, 7274, 1993,  ...,    0,    0,    0],
        ...,
        [ 101, 3636, 4031,  ..., 6316, 5206,  102],
        [ 101,  523, 6303,  ...,    0,    0,    0],
        [ 101, 7376, 1103,  ...,    0,    0,    0]])
------------------------
segments_tensors.shape = torch.Size([64, 38])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
------------------------
masks_tensors.shape    = torch.Size([64, 38])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
------------------------
label_ids.shape        

# Step3. 在BERT之上加入新layer成下游任務模型

我們是使用【單一句子分類任務】
`bertForSequenceClassification`

In [ ]:
# 載入一個可以做中文分類任務的模型，n_class = 3
from transformers import BertForSequenceClassification

PRETRAINED_MODEL_NAME = "bert-base-chinese"
NUM_LABELS = 3

model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels = NUM_LABELS)

clear_output()

# high-level 顯示此模型裡的 modules
print("""
name            module
------------------------------""")

for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))


name            module
------------------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=3, bias=True)


沒錯，一行程式碼就初始化了一個可以用 BERT 做文本多分類的模型 model。我也列出了 model 裡頭最 high level 的模組，資料流則從上到下，通過：

- BERT 處理各種 `embeddings` 的模組
- 在神經機器翻譯就已經看過的 Transformer Encoder
- 一個 pool `[CLS]` token 在所有層的 repr. 的 BertPooler
- Dropout 層
- 回傳 3 個類別 logits 的線性分類器 `classifier`

而 classifer 就只是將從 BERT 那邊拿到的 [CLS] token 的 repr. 做一個線性轉換而已，非常簡單。我也將我們實際使用的分類模型 BertForSequenceClassification 實作簡化一下供你參考：

In [ ]:
# class BertForSequenceClassification(BertPreTrainedModel):
#     def __init__(self, config, num_labels=2, ...):
#         super(BertForSequenceClassification, self).__init__(config)
#         self.num_labels = num_labels
#         self.bert = BertModel(config, ...)  # 載入預訓練 BERT
#         self.dropout = nn.Dropout(config.hidden_dropout_prob)
#         # 簡單 linear 層
#         self.classifier = nn.Linear(config.hidden_size, num_labels)
#           ...

#     def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None, ...):
#         # BERT 輸入就是 tokens, segments, masks
#         outputs = self.bert(input_ids, token_type_ids, attention_mask, ...)
#         ...
#         pooled_output = self.dropout(pooled_output)
#         # 線性分類器將 dropout 後的 BERT repr. 轉成類別 logits
#         logits = self.classifier(pooled_output)

#         # 輸入有 labels 的話直接計算 Cross Entropy 回傳，方便！
#         if labels is not None:
#             loss_fct = CrossEntropyLoss()
#             loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
#             return loss
#         # 有要求回傳注意矩陣的話回傳
#         elif self.output_attentions:
#             return all_attentions, logits
#         # 回傳各類別的 logits
#         return logits

In [ ]:
model.config

BertConfig {
  "_name_or_path": "bert-base-chinese",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

有了 model 以及我們在前一節建立的 trainloader，讓我們寫一個簡單函式測試現在 model 在訓練集上的分類準確率：

定義一個可以針對特定 DataLoader 取得模型預測結果以及分類準確度的函式
之後也可以用來生成上傳到 Kaggle 競賽的預測結果

2019/11/22 更新：在將 `tokens`、`segments_tensors` 等 tensors
丟入模型時，強力建議指定每個 tensor 對應的參數名稱，以避免 HuggingFace
更新 repo 程式碼並改變參數順序時影響到我們的結果。

In [ ]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions
    
# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
_, acc = get_predictions(model, trainloader, compute_acc=True)
print("classification acc:", acc)

device: cuda:0
classification acc: 0.995221843003413


準確率只有11%上下，這是因為我們新加的線性分類器才剛被初始化

來計算看看整個分類模型以及裏頭的簡單分類器有多少參數

In [ ]:
def get_learnable_params(module):
    return [p for p in module.parameters() if p.requires_grad]

model_params = get_learnable_params(model)
clf_params = get_learnable_params(model.classifier)

print(f"""
整個分類模型的參數量: {sum(p.numel() for p in model_params)}
線性分類器的參數量: {sum(p.numel() for p in clf_params)}
""")


整個分類模型的參數量: 102269955
線性分類器的參數量: 2307



我們新增的classifier參數量之於BERT根本滄海一粟。而因為分類模型大多數的參數都是從已訓練的 BERT 來的，實際上我們需要從頭訓練的參數量非常之少，這也是遷移學習的好處。

# Step4. 訓練該下游任務模型

記得我們前面定義的 batch 數據格式以外，訓練分類模型 model 就跟一般你使用 PyTorch 訓練模型做的事情相同。

In [ ]:
%%time

# 訓練模式
model.train()

# 使用Adam Optim更新整個分類模型的參數
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-5)

EPOCHS = 6 # 可變動
for epoch in range(EPOCHS):
    running_loss = 0.0
    for data in trainloader:
        tokens_tensors, segments_tensors, \
        masks_tensors, labels = [t.to(device) for t in data] 

        # 將參數梯度歸零
        optimizer.zero_grad()

        # forward pass
        outputs = model(input_ids = tokens_tensors,
                 token_type_ids = segments_tensors,
                 attention_mask = masks_tensors,
                 labels = labels)
        
        loss = outputs[0]
        # backword
        loss.backward()
        optimizer.step()

        # 紀錄當前 batch loss
        running_loss += loss.item()

    # 計算分類準確率
    _, acc = get_predictions(model, trainloader, compute_acc = True)
                             
    print("[epoch %d] loss: %.3f, acc: %.3f" %
        (epoch + 1, running_loss, acc))


[epoch 1] loss: 2.964, acc: 1.000
[epoch 2] loss: 0.338, acc: 1.000
[epoch 3] loss: 0.185, acc: 1.000
[epoch 4] loss: 0.117, acc: 1.000
[epoch 5] loss: 0.082, acc: 1.000
[epoch 6] loss: 0.055, acc: 1.000
CPU times: user 3min 38s, sys: 1min 26s, total: 5min 4s
Wall time: 5min 9s


# Step5. 對新樣本作推論

In [ ]:
# 整理測試集->把之前抓的農場新聞拿來測試
df_test = pd.read_csv("/content/drive/Shareddrives/Fake_news_detector/datasets_fn/PTT01/-100.csv", encoding = "utf-8")
df_test.head()

,Headline,Link,Date,Content
0,戰勝新冠肺炎了！蛇精男消失半年「無濾鏡現身」網驚：越來越好看了,https://www.ptt01.net/ptt/34380,日期：2021-06-17,大陸網紅「劉梓晨」因為誇張五官、超尖錐子臉而走紅，被網友稱作「蛇精男」，一向對自己外表很有自...
1,抖音甜美妹「失去濾鏡」秒變大媽！網哀號：還有什麼能相信,https://www.ptt01.net/ptt/34377,日期：2021-06-17,「太久保佳代子」是日本著名女諧星，也是搞笑團體「綠洲二人組」的成員之一，近日太久保佳代子在抖...
2,林萱直播邀粉絲「突現遛鳥俠」嚇壞急道歉：造成恐慌和不舒服很抱歉,https://www.ptt01.net/ptt/34374,日期：2021-06-17,YouTuber「林萱」擁有甜美長相與纖細身材，時常會分享穿搭、美食等類型影片，在網路上小有...
3,遭網嗆「不男不女所以氣死」！小A辣深夜淚憶亡父：人命真的很脆弱,https://www.ptt01.net/ptt/34372,日期：2021-06-17,網紅「小A辣」個性直率又樂觀，2020年遠赴泰國進行性別重置手術，蛻變成真正的女兒身，勇敢追...
4,帥翻動漫展引爆動！五條悟「超正」真面目曝！網讚：真的從漫畫走出來,https://www.ptt01.net/ptt/34371,日期：2021-06-16,\n「Cosplay」是在全世界都盛行許久的二次元動漫文化，動漫迷時常會透過打扮成心儀角色來...


In [ ]:
# 挑選欄位
df_test = df_test.loc[:,["Headline", "Content", "Link"]]
df_test.to_csv("/content/drive/Shareddrives/Media Framing/datasets/test_ptt01.tsv", sep = "\t", index = False)

print("預測樣本數:", len(df_test))
df_test.head()
# len(df_test)

預測樣本數: 1709


,Headline,Content,Link
0,戰勝新冠肺炎了！蛇精男消失半年「無濾鏡現身」網驚：越來越好看了,大陸網紅「劉梓晨」因為誇張五官、超尖錐子臉而走紅，被網友稱作「蛇精男」，一向對自己外表很有自...,https://www.ptt01.net/ptt/34380
1,抖音甜美妹「失去濾鏡」秒變大媽！網哀號：還有什麼能相信,「太久保佳代子」是日本著名女諧星，也是搞笑團體「綠洲二人組」的成員之一，近日太久保佳代子在抖...,https://www.ptt01.net/ptt/34377
2,林萱直播邀粉絲「突現遛鳥俠」嚇壞急道歉：造成恐慌和不舒服很抱歉,YouTuber「林萱」擁有甜美長相與纖細身材，時常會分享穿搭、美食等類型影片，在網路上小有...,https://www.ptt01.net/ptt/34374
3,遭網嗆「不男不女所以氣死」！小A辣深夜淚憶亡父：人命真的很脆弱,網紅「小A辣」個性直率又樂觀，2020年遠赴泰國進行性別重置手術，蛻變成真正的女兒身，勇敢追...,https://www.ptt01.net/ptt/34372
4,帥翻動漫展引爆動！五條悟「超正」真面目曝！網讚：真的從漫畫走出來,\n「Cosplay」是在全世界都盛行許久的二次元動漫文化，動漫迷時常會透過打扮成心儀角色來...,https://www.ptt01.net/ptt/34371


### 測試

In [ ]:
%%time
# 建立測試集。這邊我們可以用跟訓練時不同的 batch_size，看你 GPU 多大
testset = media_framing_datasets("test", tokenizer=tokenizer)
testloader = DataLoader(testset, batch_size=256, 
                        collate_fn=create_mini_batch)

# 用分類模型預測測試集
predictions = get_predictions(model, testloader)

# 用來將預測的 label id 轉回 label 文字
index_map = {v: k for k, v in testset.label_map.items()}

# 生成 Kaggle 繳交檔案
df = pd.DataFrame({"Category": predictions.tolist()})
df['Category'] = df.Category.apply(lambda x: index_map[x])
df_pred = pd.concat([testset.df.loc[:, ["Headline"]], 
                          df.loc[:, 'Category']], axis=1)
df_pred.to_csv('bert_1_prec_training_samples.csv', index=False)
predictions

CPU times: user 11 s, sys: 20.7 ms, total: 11 s
Wall time: 12.2 s


In [ ]:
df_pred = pd.read_csv("/content/bert_1_prec_training_samples.csv", encoding = "utf-8")
df_pred.head()

,Headline,Category
0,外資報告》摩根大通喊加碼聯發科 目標價1200元,economy
1,今年第14起石虎路殺 西濱後龍段石虎被撞到血肉模糊,economy
2,開始刷存在感？這些行為看出12星座想復合,economy
3,屏東潮州「賽神蝦」活動 347公克巨大泰國蝦破紀錄奪冠,economy
4,警告北約 俄稱或被迫部署中程核飛彈,economy
